In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

# New England Beer Distributors (NERD 2)
Please refer to the material of week 1 of course for more details about this problem.
## Problem Objective
Where should NERD set up its distribution Centers to serve 3 cities in order to minimiza transportation costs?

## Input Data

In [2]:
cities = ["BO", "BR", "CO", "HA", "MN", "NA", "NH", "NL", "PO", "PR", "SP", "WO"]
cities_names = ["Boston", "Brattleboro", "Concord", "Hartford", "Manchester", 
               "Nashua", "New_Haven", "New_London", "Portsmouth", "Providence", "Springfield", "Worcester"]
cities_demands = [425, 12, 43, 125, 110, 86, 129, 28, 66, 320, 220, 182]

# Creating a dataframe of cities with their corresponding names and demands.
df_cities = pd.DataFrame({
    "id": cities,
    "name": cities_names,
    "demand": cities_demands
})
print(df_cities)

candidates = ["BO", "NA", "PR", "SP", "WO"]

# Defining a distance matrix representing distances between cities (lines according to the vector cities) 
#and candidates (columns according to the vector of candidates)
distance_matrix = [[0, 37, 42, 82, 34],
[93, 65, 106, 59, 68],
[69, 33, 105, 101, 72],
[98, 103, 73, 27, 66],
[55, 20, 92, 93, 60],
[37, 0, 72, 79, 41],
[128, 137, 94, 63, 98],
[95, 113, 57, 57, 71],
[62, 48, 104, 127, 85],
[42, 72, 0, 68, 38],
[82, 79, 68, 0, 47],
[34, 41, 38, 47, 0]
]


fi = 10000    # fixed cost
cij = 1     #dolar per mile
S = 2000    # Supply max used in excel example

#defining total number of cities and candidates
num_cities = len(cities)
num_candidates = len(candidates)

    id         name  demand
0   BO       Boston     425
1   BR  Brattleboro      12
2   CO      Concord      43
3   HA     Hartford     125
4   MN   Manchester     110
5   NA       Nashua      86
6   NH    New_Haven     129
7   NL   New_London      28
8   PO   Portsmouth      66
9   PR   Providence     320
10  SP  Springfield     220
11  WO    Worcester     182


### Single Location Weber Problem 

In [3]:
# Create a solver instance
solver = pywraplp.Solver.CreateSolver('SCIP')

In [4]:
# Declare variables
x = {}  # Variables for transportation between the arc candidates to cities. 
y = {}  # Variables to represent whether a candidate city is open or not (binary).

for i in range(num_candidates):
    for j in range(num_cities):
        x[i,j] = solver.NumVar(0, solver.infinity(), "") # already implementing the non-negativity constraint
    y[i] = solver.BoolVar("")

In [5]:
M = S # M is a very big number in this case equals to the total supply

In [6]:
#Declare Constraints
#Supply limit 
for i in range(num_candidates):
    sum_value = 0
    for j in range(num_cities):
        sum_value += x[i,j]
    solver.Add(sum_value <= S)
    
# Demand requirement
for j in range(num_cities):
    solver.Add(solver.Sum([x[i,j] for i in range(num_candidates)]) >= df_cities.demand[j])

# Only one candidate city should be open.
solver.Add(solver.Sum([y[i] for i in range(num_candidates)]) == 1)

# linking constraints
for i in range(num_candidates):
    for j in range(num_cities):
        solver.Add(x[i,j] - M*y[i]  <= 0 )

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 78


In [7]:
# Calculate the fixed cost if a DC is open at location i
fixed_cost = solver.Sum([ fi * y[i]
                            for i in range(num_candidates)])

#Calculate transportation cost. Cost c_ij * distance in this example
transportation_cost = solver.Sum([ x[i,j] * distance_matrix[j][i] * cij
                            for i in range(num_candidates)
                           for j in range(num_cities)])
# Setting the objective function to minimize the total cost
solver.Minimize( transportation_cost + fixed_cost)

In [8]:
# Solve the problem
status = solver.Solve()

In [9]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solver.Objective().Value()} per week')
    for i in range(num_candidates):
        if y[i].solution_value() > 0:
            print(f'Open DC in {candidates[i]}' )  
else:
    print('No solution found.')
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())

Total cost = 89478.0 per week
Open DC in WO

Advanced usage:
Problem solved in 147.000000 milliseconds
Problem solved in 26 iterations
